In [1]:
# DESCRIPTION

# Variables that can be changed
# 1. size of smoothing kernel (default = 21)
# 2. scale factor value, used to maintain precision in data format conversion (default max_val = 1000)
# 3. smoothing factor value, used to smooth spline (default = 1200)

In [1]:
# libraries 
import nibabel as nib
import numpy as np
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
from __future__ import division
import sys
import subprocess, shlex
from scipy import ndimage

In [2]:
# definitions
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

def rescale(values, new_min = 0, new_max = 1):
    output = []
    old_min, old_max = min(values), max(values)
    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)
    return output

In [7]:
# insert filepath to folder with files
filepath = "/.../"

In [5]:
# 1. NORMALISATION OF BOTH MODELS

In [ ]:
# of model with contrast 1
input_im = filepath + "model_contrast1.mnc"
output_im = filepath + "contrast1_norm.mnc"
shell_cmd = "mincnorm" + " " + input_im + " " + output_im
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args)

# of model with contrast 2
input_im = filepath + "model_contrast2_roughAligned2contrast1.mnc"
output_im = filepath + "contrast2_roughAligned2contrast1_norm.mnc"
shell_cmd = "mincnorm" + " " + input_im + " " + output_im
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args)

In [8]:
# 2. MASK GENERATION 

In [ ]:
# format-conversion mnc2nii 
input_im =  filepath + "contrast1_norm.mnc" 
output_im = filepath + "contrast1.nii"
shell_cmd = "mnc2nii" + " " + input_im + " " + output_im 
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args) 

# bet
input_im = filepath + "contrast1.nii"
output_im = filepath + "contrast1_bet.nii"
shell_cmd = "bet" + " " + input_im + " " + output_im + " " + "-R -m -f 0.5 -v"
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args) 

# format-conversion nii2mnc 
input_im = filepath + "contrast1_bet_mask.nii"
output_im = filepath + "contrast1_bet_mask.mnc"
shell_cmd = "nii2mnc" + " " + input_im + " " + output_im
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args) 

In [14]:
# 3. RESAMPLING

In [ ]:
# resample one model (with contrast 1) to the other model (with contrast 2)
resampleToModel = filepath + "contrast2_roughAligned2contrast1_norm.mnc"
input_im = filepath + "contrast1_norm.mnc"
output_im = filepath + "contrast1_norm_resampled2contrast2.mnc"
shell_cmd = "mincresample -like" + " " + resampleToModel + " " + input_im + " " + output_im
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args)

# resampling of mask
input_im = filepath + "contrast1_bet_mask.mnc"
output_im =  filepath + "contrast1_bet_mask_resampled2contrast2.mnc"
shell_cmd = "mincresample -like" + " " + resampleToModel + " " + input_im + " " + output_im
args = shlex.split(shell_cmd)
p = subprocess.Popen(args) 

In [17]:
# 4. LOADING 

In [25]:
# load images
contrast1 = nib.load(filepath + 'contrast1_norm_resampled2contrast2.mnc')
contrast2 = nib.load(filepath + 'contrast2_roughAligned2contrast1_norm.mnc')
# load mask
mask = nib.load(filepath + 'contrast1_bet_mask_resampled2contrast2.mnc')

# get image data 
contrast1Data = contrast1.get_data()
contrast2Data = contrast2.get_data()
maskData = mask.get_data()

# convert to numpy array
arr_contrast1 = np.array(contrast1Data)
arr_contrast2 = np.array(contrast2Data)
arr_mask = np.array(maskData)

In [ ]:
# 5. APPLICATION OF MASK 

In [ ]:
contrast1_masked = arr_contrast1 * arr_mask
contrast2_masked = arr_contrast2 * arr_mask

In [ ]:
# 6. SMOOTHING OF MODEL WITH CONTRAST 1

In [13]:
contrast1_smoothed = ndimage.uniform_filter(contrast1_masked, size=[21, 21, 21])

In [ ]:
# 7. INTENSITY LOOKUP 

In [14]:
# convert from 3D to 1D
reshape_size_contrast1 = contrast1_smoothed.size
reshape_size_contrast2 = contrast2_masked.size

arr1D_contrast1 = np.reshape(contrast1_smoothed.data, reshape_size_contrast1)
arr1D_contrast2 = np.reshape(contrast2_masked.data, reshape_size_contrast2)

# max val 
max_contrast2 = np.amax(arr1D_contrast2)
max_val = 1000.0

# scaling   
scaleFactor_2 = max_val / max_contrast2
arr1D_contrast2_scaled = arr1D_contrast2 * scaleFactor_2

# convert data type (float64 --> uint32)
contrast2_uint32 = np.uint32(arr1D_contrast2_scaled)

# unique val and indeces of modality 2 
unique, indeces = np.unique(contrast2_uint32, return_index=True)

# only picking out unique val 
contrast2_unique = contrast2_uint32[indeces]

# only including unique values above 0 
M = np.where(contrast2_unique > 0)
contrast2_unique_zero = contrast2_unique[M] 

# allocating space - FIND UD AF OM ALLE VAR SKAL VAERE ARRAYS
targetVal = np.array([])
logicMatch = np.array([])
valMatch_modal1 = np.array([])
finalValMatch = np.array([])
len_contrast2_unique_zero = len(contrast2_unique_zero)

In [ ]:
tic()
# match intensities
finalValMatch = np.array([])
for i in range(len_contrast2_unique_zero):
    targetVal = contrast2_unique_zero[i]
    L = np.where(contrast2_uint32 == targetVal)
    valMatch_modal1 = arr1D_contrast1[L]
    
    Blist = stats.itemfreq(valMatch_modal1).tolist()
    
    # finding the values with the highest frequency 
    max_count = max(Blist, key=lambda x: x[1])
    max_val_list = [x for x in Blist if x[1] == max_count[1]]
    max_vals = [l[0] for l in max_val_list]
    mean_val = np.mean(max_vals)
    finalValMatch = np.append(finalValMatch, mean_val)
toc()

In [20]:
# data type conversion and rescaling of modality 2
contrast2_unique_fl64 = np.float64(contrast2_unique_zero) 
contrast2_unique_fl64_rescaled = contrast2_unique_fl64 / scaleFactor_2

In [30]:
# 8. SPLINE FITTING

In [21]:
x = contrast2_unique_fl64_rescaled # intensities of contrast 2
y = finalValMatch # matching intensities of contrast 1

plt.plot(x, y, '.', label="W21 data points") 

# creating the spline
spl = UnivariateSpline(x, y)
# adjusting the amount of smoothing:
spl.set_smoothing_factor(1200)

x_converted = spl(x)
plt.plot(x, x_converted, 'g', lw=1, label="Fitted spline (sf = 1200)")  
plt.legend(loc='upper right')
plt.xlabel('Intensity values of TSE image volume')
plt.ylabel('Intensity values of MP2RAGE volume')
plt.show()

In [33]:
# 9. LOOKUP TABLE GENERATION

In [ ]:
firstLutColumn = x
secondLutColumn = x_converted

# rescaling of intensity values to the range 0-1
firstLutColumn = rescale(firstLutColumn, 0, 1)
secondLutColumn = rescale(secondLutColumn, 0, 1)

# saving lookup table (lut) as .txt
lut = open("lookuptable.txt","w")
for j in range(len(firstLutColumn)):
    firstLutColumn_str = str(firstLutColumn[j])
    secondLutColumn_str = str(secondLutColumn[j])
    lut.write(firstLutColumn_str + " " + secondLutColumn_str + "\n")
lut.close()

In [36]:
# 10. CONTRAST CONVERSION OF MODEL WITH CONTRAST 2 USING LOOKUP TABLE

In [ ]:
# conversion using look up function
lut = filepath + "lookuptable.txt"
input_im = filepath + "model_contrast2_roughAligned2contrast1.mnc"
output_im = filepath + "contrast2_lookupConverted2contrast1.mnc"
shell_cmd = "minclookup -continuous -lookup_table" + " " + lut + " " + input_im + " " + output_im + " " + "-2"
args = shlex.split(shell_cmd)
print(args)
p = subprocess.Popen(args) 